In [1]:
import numpy as np

In [11]:
# first, define our problem size
n = 10
alpha = np.random.random((n, 1))
sigma = np.random.random((n, 1))
k = np.random.random((n, n))

# goal, generate some vector r such that:
# sum(r) = 1.0
# r represents our optimal investment portfolio


In [12]:
# element-wise math

# square every term in alpha
alpha_element_squared = alpha * alpha
sigma_element_squared = sigma * sigma

# the 3.9 formula from
# https://maslov.bioengineering.illinois.edu/optimal_investment_ijtaf.pdf
alpha / (alpha_element_squared + sigma_element_squared)


array([[1.0125045 ],
       [0.91344153],
       [0.62399147],
       [0.98555817],
       [0.48416987],
       [0.77845226],
       [1.04135925],
       [1.25660972],
       [0.625922  ],
       [2.25253198]])

In [16]:
# invert matrices
k_inv = np.linalg.inv(k)

# formula 3.10 from 
# https://maslov.bioengineering.illinois.edu/optimal_investment_ijtaf.pdf
r_opt = np.dot(k_inv, alpha)


In [17]:
# hints that I'm not sure how they all go together

# from formula 3.11
# you know that if \lambda >= \alpha_i, then we can ignore that term

# Maybe if you sort all fo the \alpha_i terms, you can see if a solution for lambda exists where all of the smaller
# values are ignored, and all of the larger values are effected by \lambda

# if you look at each term in the sum of 3.12, they are piecewise linear wrt. \lambda


array([[-1.51610396],
       [-0.2963087 ],
       [-0.88903972],
       [-1.42098409],
       [ 2.41201645],
       [-2.23738907],
       [-0.36933364],
       [ 3.61882801],
       [ 1.68998735],
       [ 0.76383406]])